In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn import preprocessing
from keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D,Flatten,MaxPooling2D,Dropout,MaxPool2D
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import os
import pprint
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model

In [ ]:
path = "C:/Users/Lisa/Rezanur/Paper 1/Test 2/Wavelets/"

In [ ]:
Concat_Spec = pd.read_csv(path + "Tesla_Train_Wavelet_Bin.csv")
Concat_Spec.shape

In [ ]:
ss = Concat_Spec.shape[1]
stamp = 10

In [ ]:
Final = np.array(Concat_Spec)
Final = Final[:120300]
print(Final.shape)

In [ ]:
X = Final[:, :ss-1]
y = Final[:,ss-1]

In [ ]:
#Data normalization by scalling
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X.shape

In [ ]:
pd.value_counts(y)

In [ ]:
X_trainX = []
y_trainX = []
strat_time = 0
timestamp = 10
length = len(Final)

for i in range(length):
    X_trainX.append(X[strat_time:timestamp])
    y_trainX.append(y[timestamp])
    
    if timestamp >= length-10:
        break
    
    strat_time = strat_time + 1
    timestamp = timestamp + 1
print(len(X_trainX))
print(len(y_trainX))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_trainX, y_trainX, test_size=0.10, random_state=0)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
pd.value_counts(y_train)

In [ ]:
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
print(y_train[0])
print(y_test[4])

In [ ]:
print(X_train.shape)

In [ ]:
#Resape data into 3 dimention
X_train = np.reshape(X_train, (X_train.shape[0], stamp, ss-1, 1))
X_test = np.reshape(X_test, (X_test.shape[0], stamp, ss-1, 1))

In [ ]:
print(X_train[0])
print('\n')
print(y_train[0])

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='leaky_relu',padding='same',input_shape=(stamp, ss-1, 1)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.1))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.1))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(LeakyReLU(alpha=0.1))           
model.add(Dropout(0.1))
model.add(Dense(2, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['acc'])

model.summary()

In [ ]:
cp = tf.keras.callbacks.ModelCheckpoint(filepath="Wavelet_BIN_CNN_Tesla.h5",
                               mode='min', monitor='val_loss', verbose=0, save_best_only=True)
# define our early stopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=10,
    verbose=1, 
    mode='min',
    restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=1250,
                    batch_size=64,
                    shuffle=False,
                    validation_data=(X_test, y_test),
                    verbose=1,
                    callbacks=[cp, early_stop]
                    ).history

In [ ]:
# plot the training losses
fig, ax = plt.subplots(figsize=(8, 6), dpi=80)
ax.plot(model.history.history['loss'], 'b', label='Train', linewidth=2)
ax.plot(model.history.history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

In [ ]:
Tesla_Normal_Spec = pd.read_csv(path + "Tesla_Test_Wavelet_Bin.csv")
Tesla_Normal_Spec.shape

In [ ]:
tesla = np.array(Tesla_Normal_Spec)

In [ ]:
X = tesla[:, :ss-1]
y = tesla[:,ss-1]

In [ ]:
#Data normalization by scalling
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X.shape

In [ ]:
X_test = []
y_test = []
strat_time = 0
timestamp = 10
length = len(tesla)

for i in range(length):
    X_test.append(X[strat_time:timestamp])
    y_test.append(y[timestamp])
    
    if timestamp >= length-10:
        break
    
    strat_time = strat_time + 1
    timestamp = timestamp + 1
print(len(X_test))
print(len(y_test))

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
#Resape data into 3 dimention
X_test = np.reshape(X_test, (X_test.shape[0], stamp, ss-1, 1))

In [ ]:
y_test = to_categorical(y_test, 2)
y_test[0]

In [ ]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('Wavelet_BIN_CNN_Tesla.h5')

In [ ]:
#Prediction
pred_x = model.predict(X_test)
#obj = scaler.fit(y_pred)
#predicted = obj.inverse_transform(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = np.argmax(pred_x, axis=1)
# label
y_test = np.argmax(y_test, axis=1)
#Get the confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred)

print(cf_matrix)

In [ ]:
#group_names = ['True Neg','False Pos','False Neg','True Pos','True Pos','True Pos','True Pos','True Pos','True Pos']

group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]

group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]

labels = [f"{v1}\n{v2}\n" for v1, v2 in
          zip(group_counts,group_percentages)]

labels = np.asarray(labels).reshape(2,2)
sns.set(rc={'figure.figsize':(15,13)})
sns.set(font_scale=1.8) #edited as suggested
ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues', cbar=False)


ax.set_xlabel('\n Actual Category', fontname="Times New Roman", size=22,fontweight="bold")
ax.set_ylabel('Predicted Category \n', fontname="Times New Roman", size=22,fontweight="bold");
#ax.text(0.2, 2.30, 'Fig. 9. (c) Confusion Matrix CNN Multiclass Classification',fontsize=25,fontweight="bold")

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(["Normal", "Attack"],fontweight="bold", fontname="Times New Roman", size=18)
ax.yaxis.set_ticklabels(["Normal", "Attack"],fontweight="bold", fontname="Times New Roman", size=18)
ax.xaxis.label.set_color('darkblue')
ax.yaxis.label.set_color('darkblue')
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
from sklearn.metrics import classification_report
class_names = ["Normal", "Attack"]
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(15,10)})
sns.set(font_scale=1.3) #edited as suggested

plt.plot(false_positive_rate,true_positive_rate, color='red',label = 'BIN_CNN AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1])

plt.xlabel('False Positive Rate(TPR)', fontname="Times New Roman", size=18, fontweight="bold", color='darkblue')
plt.ylabel('True Positive Rate (FPR)', fontname="Times New Roman", size=18, fontweight="bold", color='darkblue')
plt.text(0.02, -0.22, 'Fig. 10. (c) RUC Score CNN Binary Class Classification Vehicle 4', fontname="Times New Roman",fontsize=25,fontweight="bold")
plt.legend(loc="lower right")